<a id="readme-top"></a>
# LP read mps notebook

## 📚 Table of Content
- [Reading File to String](#reading-file-to-string)
- [Reading MPS File From String](#reading-mps-file-from-string)
    - [Original Function](#original-function)
    - [Adding Multiple Column Support](#adding-multiple-column-support)
- [Unit Testing](#unit-testing)
    - [Manual Approach](#manual-approach)
- [Testing with JuMP](#testing-with-jump)
    - [Using JuMP MathOptInterface as a reference for testing](#using-jump-mathoptinterface-as-a-reference-for-testing)


In [16]:
using LinearAlgebra
using SparseArrays
using Random
using ArgParse
using DataStructures
using Test

# JuMP
using MathOptInterface
const MOI = MathOptInterface

# local modules
push!(LOAD_PATH, realpath("../code"))
using lp_constants
using lp_utils
using lp_problem



<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Reading File to String

The following function reads a file and outputs a string. It is an important step because we can use string methods and technequies to access data.

In [4]:
function read_file_to_string(file_path::String)
    file_string = open(file_path, "r") do f
        read(f, String)
    end
    return file_string
end

read_file_to_string (generic function with 1 method)

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Reading MPS File From String

In [ ]:
function read_mps_from_file(file_path::String)
    mps_string = open(file_path, "r") do f
        read(f, String)
    end
    return read_mps_from_string(mps_string)
end

<p align="right">(<a href="#readme-top">back to top</a>)</p>

### Original Function

In [ ]:
function read_mps_from_string_multicolumns(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => OrderedDict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true

    objective_set = false
    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
                objective_set = true
            elseif words[1] == "MIN"
                is_minimize = true
                objective_set = true
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            col_name = words[1]
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = OrderedDict()
            end

            row_name_1, value_1 = words[2:3]
            sections["COLUMNS"][col_name][row_name_1] = parse(Float64, value_1)
            
            if length(words) > 3
                row_name_2, value_2 = words[4:5]
                sections["COLUMNS"][col_name][row_name_2] = parse(Float64, value_2)
            end
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            if length(words) == 4  # LO, UP, and FX
                bound_type, _, var_name, value = words
            elseif length(words) == 3  # FR
                bound_type, _, var_name = words
                value = Inf
            end
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end
            if bound_type == "FR"
                sections["BOUNDS"][var_name][bound_type] = nothing
            else
                sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
            end
        end
    end

    # Convert to LPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = spzeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
            if haskey(bounds, "FR")
                lb[i] = -Inf
                ub[i] = Inf
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    return LPProblem(is_minimize, c, A, b, lb, ub, vars, constraint_types)
end

### Adding Multiple Column Support

In [5]:
function read_mps_from_string_multicolumns(mps_string::String)
    lines = split(mps_string, '\n')
    sections = Dict("NAME" => "", "ROWS" => [], "COLUMNS" => OrderedDict(), "RHS" => Dict(), "BOUNDS" => Dict())
    current_section = ""
    objective_name = ""
    is_minimize = true

    objective_set = false
    for line in lines
        words = split(line)
        (isempty(words) || (line[1] == '*')) && continue  # Skip empty lines and comments

        if (line[1] != ' ') && words[1] in ["NAME", "OBJSENSE", "ROWS", "COLUMNS", "RHS", "BOUNDS", "ENDATA"]
            current_section = words[1]
            continue
        end

        if current_section == "NAME"
            sections["NAME"] = words[1]
        elseif current_section == "OBJSENSE"
            if words[1] == "MAX"
                is_minimize = false
                objective_set = true
            elseif words[1] == "MIN"
                is_minimize = true
                objective_set = true
            end
        elseif current_section == "ROWS"
            row_type, row_name = words
            push!(sections["ROWS"], (type=row_type, name=row_name))
            if row_type == "N"
                objective_name = row_name
            end
        elseif current_section == "COLUMNS"
            col_name = words[1]
            if !haskey(sections["COLUMNS"], col_name)
                sections["COLUMNS"][col_name] = OrderedDict()
            end

            row_name_1, value_1 = words[2:3]
            sections["COLUMNS"][col_name][row_name_1] = parse(Float64, value_1)
            
            if length(words) > 3
                row_name_2, value_2 = words[4:5]
                sections["COLUMNS"][col_name][row_name_2] = parse(Float64, value_2)
            end
        elseif current_section == "RHS"
            if length(words) == 3
                _, row_name, value = words
            else
                row_name, value = words[2:3]
            end
            sections["RHS"][row_name] = parse(Float64, value)
        elseif current_section == "BOUNDS"
            if length(words) == 4  # LO, UP, and FX
                bound_type, _, var_name, value = words
            elseif length(words) == 3  # FR
                bound_type, _, var_name = words
                value = Inf
            end
            if !haskey(sections["BOUNDS"], var_name)
                sections["BOUNDS"][var_name] = Dict()
            end
            if bound_type == "FR"
                sections["BOUNDS"][var_name][bound_type] = nothing
            else
                sections["BOUNDS"][var_name][bound_type] = parse(Float64, value)
            end
        end
    end

    # Convert to LPProblem structure
    vars = collect(keys(sections["COLUMNS"]))
    n_vars = length(vars)
    n_constraints = count(row -> row.type != "N", sections["ROWS"])

    c = zeros(n_vars)
    A = spzeros(n_constraints, n_vars)
    b = zeros(n_constraints)
    constraint_types = Char[]

    # Populate objective function
    for (i, var) in enumerate(vars)
        if haskey(sections["COLUMNS"][var], objective_name)
            c[i] = sections["COLUMNS"][var][objective_name]
        end
    end

    # Populate constraint matrix and right-hand side
    constraint_index = 0
    for row in sections["ROWS"]
        if row.type != "N"
            constraint_index += 1
            push!(constraint_types, row.type[1])  # Store constraint type
            for (i, var) in enumerate(vars)
                if haskey(sections["COLUMNS"][var], row.name)
                    A[constraint_index, i] = sections["COLUMNS"][var][row.name]
                end
            end
            b[constraint_index] = get(sections["RHS"], row.name, 0.0)
            
            # Adjust for 'G' type constraints
            if row.type == "G"
                A[constraint_index, :] *= -1
                b[constraint_index] *= -1
            end
        end
    end

    # Process bound constraints
    lb = fill(-Inf, n_vars)
    ub = fill(Inf, n_vars)
    for (i, var) in enumerate(vars)
        if haskey(sections["BOUNDS"], var)
            bounds = sections["BOUNDS"][var]
            if haskey(bounds, "LO")
                lb[i] = bounds["LO"]
            end
            if haskey(bounds, "UP")
                ub[i] = bounds["UP"]
            end
            if haskey(bounds, "FX")
                lb[i] = ub[i] = bounds["FX"]
            end
            if haskey(bounds, "FR")
                lb[i] = -Inf
                ub[i] = Inf
            end
        else
            lb[i] = 0.0  # Default lower bound is 0 if not specified
        end
    end

    return LPProblem(is_minimize, c, A, b, lb, ub, vars, constraint_types)
end

read_mps_from_string_multicolumns (generic function with 1 method)

In [8]:

# Test
mps_filename = "../../benchmarks/mps_files/test.mps"
test_string = read_file_to_string(mps_filename)

lp = read_mps_from_string_multicolumns(test_string)
println(lp)

println("")

dense_lp = Matrix(lp.A)
println("c: ", lp.c)
println("b: ", lp.b)
println("vars: ", lp.vars)
println("constraint_types: ", lp.constraint_types)
println("A: ", dense_lp)

LPProblem(false, [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0], sparse([1, 3, 4, 5, 1, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 2, 3, 4, 5, 3, 4, 5], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6], [1.0, 8.8, -8.8, 1.0, 1.0, 6.1, -6.1, 1.0, 1.0, 2.0, -2.0, 1.0, 1.0, 4.2, -4.2, 1.0, 1.0, 5.0, -5.0, 1.0, -6.0, 3.0, -1.0], 5, 6), [200.0, 250.0, 0.0, -0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [Inf, Inf, Inf, Inf, Inf, Inf], ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"], ['L', 'L', 'L', 'G', 'E'])

c: [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0]
b: [200.0, 250.0, 0.0, -0.0, 0.0]
vars: ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"]
constraint_types: ['L', 'L', 'L', 'G', 'E']
A: [1.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 1.0 1.0 0.0; 8.8 6.1 2.0 4.2 5.0 -6.0; -8.8 -6.1 -2.0 -4.2 -5.0 3.0; 1.0 1.0 1.0 1.0 1.0 -1.0]


<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Unit Testing

### Manual Approach

In [10]:
mps_filename = "../../benchmarks/mps_files/test.mps"
lp_string = read_file_to_string(mps_filename)
println(lp_string)

NAME          BLEND

OBJSENSE
 MAX
 
ROWS
 N  PROF
 L  VVEG
 L  NVEG
 L  UHRD
 G  LHRD
 E  CONT
COLUMNS
    VEG01     PROF      -110.0    VVEG         1.0
    VEG01     UHRD         8.8    LHRD         8.8
    VEG01     CONT         1.0
    VEG02     PROF      -120.0    VVEG         1.0
    VEG02     UHRD         6.1    LHRD         6.1
    VEG02     CONT         1.0
    OIL01     PROF      -130.0    NVEG         1.0
    OIL01     UHRD         2.0    LHRD         2.0
    OIL01     CONT         1.0
    OIL02     PROF      -110.0    NVEG         1.0
    OIL02     UHRD         4.2    LHRD         4.2
    OIL02     CONT         1.0
    OIL03     PROF      -115.0    NVEG         1.0
    OIL03     UHRD         5.0    LHRD         5.0
    OIL03     CONT         1.0
    PROD      PROF       150.0    UHRD        -6.0
    PROD      LHRD        -3.0    CONT        -1.0
RHS
    RHS       VVEG       200.0
    RHS       NVEG       250.0
    RHS       UHRD         0.0
    RHS       LHRD         0.0
 

In [17]:
# Define the expected outputs for comparison
expected_c = [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0]
expected_b = [200.0, 250.0, 0.0, 0.0, 0.0]
expected_vars = ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"]
expected_constraint_types = ['L', 'L', 'L', 'G', 'E']

# Define the expected A matrix (as a sparse matrix for comparison)
expected_A = [
    [1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 1.0, 1.0, 0.0],
    [8.8, 6.1, 2.0, 4.2, 5.0, -6.0],
    [8.8, 6.1, 2.0, 4.2, 5.0, -3.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, -1.0]
]

# The map(x -> x == -0.0 ? 0.0 : x, lp.) is used to ensure no non zeros.

# Write the test cases
@testset "MPS Parsing Tests" begin
    # Test the objective function coefficients
    @test map(x -> x == -0.0 ? 0.0 : x, lp.c) == expected_c
    
    # Test the right-hand side vector
    @test map(x -> x == -0.0 ? 0.0 : x, lp.b) == expected_b
    
    # Test the variable names
    @test map(x -> x == -0.0 ? 0.0 : x, lp.vars) == expected_vars
    
    # Test the constraint types
    @test map(x -> x == -0.0 ? 0.0 : x, lp.constraint_types) == expected_constraint_types
    
    # Test the constraint matrix (A)
    #@test Matrix(lp.A) == expected_A # This indicates an issue.
end

Test Summary:     | Pass  Total
MPS Parsing Tests |    4      4


Test.DefaultTestSet("MPS Parsing Tests", Any[], 4, false, false)

<p align="right">(<a href="#readme-top">back to top</a>)</p>

## Testing with JuMP

Here I look at the multicolumned example of test.mps to check if the output is as expected.

In [14]:
mps_filename = "../../benchmarks/mps_files/test.mps"
test_string = read_file_to_string(mps_filename)
println(test_string)

NAME          BLEND

OBJSENSE
 MAX
 
ROWS
 N  PROF
 L  VVEG
 L  NVEG
 L  UHRD
 G  LHRD
 E  CONT
COLUMNS
    VEG01     PROF      -110.0    VVEG         1.0
    VEG01     UHRD         8.8    LHRD         8.8
    VEG01     CONT         1.0
    VEG02     PROF      -120.0    VVEG         1.0
    VEG02     UHRD         6.1    LHRD         6.1
    VEG02     CONT         1.0
    OIL01     PROF      -130.0    NVEG         1.0
    OIL01     UHRD         2.0    LHRD         2.0
    OIL01     CONT         1.0
    OIL02     PROF      -110.0    NVEG         1.0
    OIL02     UHRD         4.2    LHRD         4.2
    OIL02     CONT         1.0
    OIL03     PROF      -115.0    NVEG         1.0
    OIL03     UHRD         5.0    LHRD         5.0
    OIL03     CONT         1.0
    PROD      PROF       150.0    UHRD        -6.0
    PROD      LHRD        -3.0    CONT        -1.0
RHS
    RHS       VVEG       200.0
    RHS       NVEG       250.0
    RHS       UHRD         0.0
    RHS       LHRD         0.0
 

In [18]:
# Define the expected outputs for comparison
expected_c = [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0]
expected_b = [200.0, 250.0, 0.0, 0.0, 0.0]
expected_vars = ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"]
expected_constraint_types = ['L', 'L', 'L', 'G', 'E']

# Define the expected A matrix (as a sparse matrix for comparison)
expected_A = [
    [1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 1.0, 1.0, 1.0, 0.0],
    [8.8, 6.1, 2.0, 4.2, 5.0, -6.0],
    [8.8, 6.1, 2.0, 4.2, 5.0, -3.0],
    [1.0, 1.0, 1.0, 1.0, 1.0, -1.0]
]

# The map(x -> x == -0.0 ? 0.0 : x, lp.) is used to ensure no non zeros.

# Write the test cases
@testset "MPS Parsing Tests" begin
    # Test the objective function coefficients
    @test map(x -> x == -0.0 ? 0.0 : x, lp.c) == expected_c
    
    # Test the right-hand side vector
    @test map(x -> x == -0.0 ? 0.0 : x, lp.b) == expected_b
    
    # Test the variable names
    @test map(x -> x == -0.0 ? 0.0 : x, lp.vars) == expected_vars
    
    # Test the constraint types
    @test map(x -> x == -0.0 ? 0.0 : x, lp.constraint_types) == expected_constraint_types
    
    # Test the constraint matrix (A)
    #@test Matrix(lp.A) == expected_A
end

Test Summary:     | Pass  Total
MPS Parsing Tests |    4      4


Test.DefaultTestSet("MPS Parsing Tests", Any[], 4, false, false)

### Using JuMP MathOptInterface as a reference for testing

In [19]:
"""
    read_and_process_mps(file_path::String) -> LPProblem

Reads an MPS file from the provided file path, processes it into a MathOptInterface model, 
and returns an LPProblem struct with all the relevant details.

# Arguments
- `file_path::String`: The file path to the MPS file.

# Returns
- `LPProblem`: A struct representing the linear programming problem.
"""
function read_mps_with_JuMP(file_path::String)
    # Create a utility model
    model = MOI.Utilities.Model{Float64}()

    # Read the MPS file into the model
    MOI.read_from_file(model, file_path)

    # Extract variables
    variables = MOI.get(model, MOI.ListOfVariableIndices())

    # Extract variable names
    variable_names = [MOI.get(model, MOI.VariableName(), var) for var in variables]

    # Extract the objective function (assumes a linear objective)
    objective_function = MOI.get(model, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}())
    objective_coeffs = zeros(Float64, length(variables))

    # Populate the objective coefficients array
    for term in objective_function.terms
        objective_coeffs[term.variable.value] = term.coefficient
    end

    # Determine whether it's a minimization or maximization problem
    is_minimize = MOI.get(model, MOI.ObjectiveSense()) == MOI.MIN_SENSE

    # Separate constraints by type
    less_than_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())
    greater_than_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.GreaterThan{Float64}}())
    equal_to_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.EqualTo{Float64}}())

    constraint_matrix_rows = Int64[]
    constraint_matrix_cols = Int64[]
    constraint_matrix_vals = Float64[]
    rhs_values = Float64[]
    constraint_types = Char[]

    # Process LessThan constraints
    for con in less_than_constraints
        func = MOI.get(model, MOI.ConstraintFunction(), con)
        set = MOI.get(model, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.upper)
        push!(constraint_types, 'L')
    end

    # Process GreaterThan constraints
    for con in greater_than_constraints
        func = MOI.get(model, MOI.ConstraintFunction(), con)
        set = MOI.get(model, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.lower)
        push!(constraint_types, 'G')
    end

    # Process EqualTo constraints
    for con in equal_to_constraints
        func = MOI.get(model, MOI.ConstraintFunction(), con)
        set = MOI.get(model, MOI.ConstraintSet(), con)
        
        for term in func.terms
            push!(constraint_matrix_rows, con.value)  # Constraint index
            push!(constraint_matrix_cols, term.variable.value)  # Variable index
            push!(constraint_matrix_vals, term.coefficient)
        end
        push!(rhs_values, set.value)
        push!(constraint_types, 'E')
    end

    # Convert to sparse matrix
    constraint_matrix = sparse(constraint_matrix_rows, constraint_matrix_cols, constraint_matrix_vals, length(rhs_values), length(variables))

    # Define lower and upper bounds
    lower_bounds = fill(0.0, length(variables))  # Default lower bounds (0.0)
    upper_bounds = fill(Inf, length(variables))  # Default upper bounds (Inf)

    # Construct the LPProblem struct
    lp = LPProblem(
        is_minimize,
        objective_coeffs,
        constraint_matrix,
        rhs_values,
        lower_bounds,
        upper_bounds,
        variable_names,
        constraint_types
    )

    return lp
end



read_mps_with_JuMP

In [21]:
# Example usage:
lp = read_mps_with_JuMP(mps_filename)

# Print the LPProblem struct
println(lp)

# Convert the constraint matrix to a dense matrix and print details
dense_lp = Matrix(lp.A)
println("c: ", lp.c)
println("b: ", lp.b)
println("vars: ", lp.vars)
println("constraint_types: ", lp.constraint_types)
println("A: ", dense_lp)

LPProblem(false, [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0], sparse([1, 3, 1, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 3], [1, 1, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6], [10.8, 8.8, 8.1, 6.1, 3.0, 1.0, 2.0, 5.2, 1.0, 4.2, 6.0, 1.0, 5.0, -4.0, -6.0], 5, 6), [200.0, 250.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [Inf, Inf, Inf, Inf, Inf, Inf], ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"], ['L', 'L', 'L', 'G', 'E'])
c: [-110.0, -120.0, -130.0, -110.0, -115.0, 150.0]
b: [200.0, 250.0, 0.0, 0.0, 0.0]
vars: ["VEG01", "VEG02", "OIL01", "OIL02", "OIL03", "PROD"]
constraint_types: ['L', 'L', 'L', 'G', 'E']
A: [10.8 8.1 3.0 5.2 6.0 -4.0; 0.0 0.0 1.0 1.0 1.0 0.0; 8.8 6.1 2.0 4.2 5.0 -6.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0]


In [22]:
function test_read_mps_from_string_multicolumns(mps_file::String)
    @testset "Test read_mps_from_string_multicolumns function with $mps_file" begin
        
        # Use the read_file_to_string function to read the MPS file content as a string
        test_string = read_file_to_string(mps_file)

        # Use the function you wrote to parse the MPS string
        lp_output = read_mps_from_string_multicolumns(test_string)

        # Use MOI to create the expected model
        model = MOI.Utilities.Model{Float64}()
        MOI.read_from_file(model, mps_file)

        # Extract variables and their names using MOI
        variables = MOI.get(model, MOI.ListOfVariableIndices())
        variable_names = [MOI.get(model, MOI.VariableName(), var) for var in variables]

        # Extract the objective coefficients using MOI
        objective_function = MOI.get(model, MOI.ObjectiveFunction{MOI.ScalarAffineFunction{Float64}}())
        expected_objective_coeffs = zeros(Float64, length(variables))
        for term in objective_function.terms
            expected_objective_coeffs[term.variable.value] = term.coefficient
        end

        # Extract the RHS and constraint matrix using MOI
        less_than_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())
        greater_than_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.GreaterThan{Float64}}())
        equal_to_constraints = MOI.get(model, MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.EqualTo{Float64}}())

        constraint_matrix_rows = Int[]
        constraint_matrix_cols = Int[]
        constraint_matrix_vals = Float64[]
        expected_rhs = Float64[]
        expected_constraint_types = Char[]

        for con in less_than_constraints
            func = MOI.get(model, MOI.ConstraintFunction(), con)
            set = MOI.get(model, MOI.ConstraintSet(), con)
            for term in func.terms
                push!(constraint_matrix_rows, con.value)
                push!(constraint_matrix_cols, term.variable.value)
                push!(constraint_matrix_vals, term.coefficient)
            end
            push!(expected_rhs, set.upper)
            push!(expected_constraint_types, 'L')
        end

        for con in greater_than_constraints
            func = MOI.get(model, MOI.ConstraintFunction(), con)
            set = MOI.get(model, MOI.ConstraintSet(), con)
            for term in func.terms
                push!(constraint_matrix_rows, con.value)
                push!(constraint_matrix_cols, term.variable.value)
                push!(constraint_matrix_vals, term.coefficient)
            end
            push!(expected_rhs, set.lower)
            push!(expected_constraint_types, 'G')
        end

        for con in equal_to_constraints
            func = MOI.get(model, MOI.ConstraintFunction(), con)
            set = MOI.get(model, MOI.ConstraintSet(), con)
            for term in func.terms
                push!(constraint_matrix_rows, con.value)
                push!(constraint_matrix_cols, term.variable.value)
                push!(constraint_matrix_vals, term.coefficient)
            end
            push!(expected_rhs, set.value)
            push!(expected_constraint_types, 'E')
        end

        expected_constraint_matrix = sparse(constraint_matrix_rows, constraint_matrix_cols, constraint_matrix_vals, length(expected_rhs), length(variables))

        # Define expected results
        expected_vars = variable_names
        expected_c = expected_objective_coeffs

        # Convert constraint matrix to dense for easier testing
        dense_expected_A = Matrix(expected_constraint_matrix)

        # Assertions to compare function output with MOI expected output
        println("Testing variable names...")
        @test lp_output.vars == expected_vars

        println("Testing objective coefficients...")
        @test lp_output.c == expected_c

        println("Testing RHS values...")
        @test lp_output.b == expected_rhs

        println("Testing constraint types...")
        @test lp_output.constraint_types == expected_constraint_types

        println("Testing constraint matrix...")
        @test Matrix(lp_output.A) == dense_expected_A

    end
end


test_read_mps_from_string_multicolumns (generic function with 1 method)

In [23]:

# Run the test with a specific MPS file
test_filepath = mps_filename
test_read_mps_from_string_multicolumns(test_filepath)

Testing variable names...
Testing objective coefficients...
Testing RHS values...
Testing constraint types...
Testing constraint matrix...
Test read_mps_from_string_multicolumns function with ../../benchmarks/mps_files/test.mps: Test Failed at In[22]:95
  Expression: Matrix(lp_output.A) == dense_expected_A
   Evaluated: [1.0 1.0 … 0.0 0.0; 0.0 0.0 … 1.0 0.0; … ; -8.8 -6.1 … -5.0 3.0; 1.0 1.0 … 1.0 -1.0] == [10.8 8.1 … 6.0 -4.0; 0.0 0.0 … 1.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
Stacktrace:
 [1] macro expansion
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Test/src/Test.jl:445 [inlined]
 [2] macro expansion
   @ ./In[22]:95 [inlined]
 [3] macro expansion
   @ /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Test/src/Test.jl:1283 [inlined]
 [4] test_read_mps_from_string_multicolumns(mps_file::String)
   @ Main ./In[22]:5
Test Summary:                                                                            | Pass

LoadError: [91mSome tests did not pass: 4 passed, 1 failed, 0 errored, 0 broken.[39m